based on [this comment][1]

  [1]: https://www.kaggle.com/c/facebook-recruiting-iii-keyword-extraction/forums/t/6650/share-your-approach?forumMessageId=36434#post36434

In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output

# Any results you write to the current directory are saved as output.

In [4]:
biology = pd.read_csv("../input/biology.csv")
cooking = pd.read_csv("../input/cooking.csv")
crypto = pd.read_csv("../input/crypto.csv")
diy = pd.read_csv("../input/diy.csv")
robotics = pd.read_csv("../input/robotics.csv")
sample_submission = pd.read_csv("../input/sample_submission.csv")
test = pd.read_csv("../input/test.csv")
travel = pd.read_csv("../input/travel.csv")

In [ ]:
import re
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer
stemmer = SnowballStemmer("english")
wordnet_lemmatizer = WordNetLemmatizer()
punctuation_trans_table = str.maketrans({key: None for key in string.punctuation})
html_tag_regex = re.compile('<.*?>')

print(biology.head())

def cleaning_text(text):
    # TODO remove code fragment
    # TODO remove url
    # TODO convert to lowercase
    # TODO add meta features from original text
    ## length of the raw text in chars
    ## number of code segments
    ## number of 'a href' tags
    ## number of times 'http' occurs (count urls)
    ## number of times 'grater sign' occurs (count html tags)
    # TODO add meta features from cleaned text
    ## number of words(tokens) in the clean text
    ## length of the clean text in chars
    # TODO feature scaling(0-1 range) with min-max
    
    # remove html tags
    text = re.sub(html_tag_regex, '', text)
    # remove \r, \n
    text = text.replace('\n', ' ').replace('\r', '')
    # remove Punctuations
    text = text.translate(punctuation_trans_table)
    # split
    words = word_tokenize(text)
    # remove stop words
    words = [word for word in words if word not in stopwords.words('english')]
    # lemmatizing, stemming
    words = [stemmer.stem(wordnet_lemmatizer.lemmatize(word)) for word in words]
    # join
    text = ' '.join(words)
    return text


def cleaning(row):
    row['title'] = cleaning_text(row['title'])
    row['content'] = cleaning_text(row['content'])
    return row

# TODO remove duplicates
# TODO union tags for duplicates
cleaned_df = biology.apply(cleaning, axis=1)
print(cleaned_df.head())


In [ ]:
print(len(cleaned_df))

In [ ]:
from sklearn.feature_extraction import text
stop_words = text.ENGLISH_STOP_WORDS

from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(stop_words=stop_words)
print(cleaned_df[['content']])
trans_word_data = vectorizer.fit_transform(cleaned_df['content'].tolist())
print(trans_word_data)

In [ ]:
# TODO extract most common tags
tags_list = cleaned_df['tags'].str.split(pat=' ').tolist()
total_tags = pd.Series([item for sublist in tags_list for item in sublist])
print(len(total_tags))
print(total_tags.value_counts())


In [ ]:
# TODO train SGD classifier with one-vs-rest approach
from sklearn.linear_model import SGDClassifier
clf = SGDClassifier(loss="modified_huber")
